In [10]:
import numpy as np
from numpy import linalg as LA
import random as ran
    


class optimization:

    def __init__ (self,fun, h):
        self.fun = fun
        self.h = h

    def grad(self, x0):
        n = len(x0)
        G = np.zeros((n,1),dtype = float)
        for i in range(n):
            xh = np.array(x0, dtype = float)
            xh[i,0] = xh[i,0] + self.h
            G[i,0] = round((self.fun(xh)-self.fun(x0))/self.h,3) 
        return G

    def hess(self, x0):
        n = len(x0)
        x0 = x0.reshape((n,))
        s= (n,n)
        H = np.zeros(s)
        for i in range(n):
            for j in range(n):
                z1 = np.zeros(n)
                z2 = z1
                z1[i] += self.h
                z2[j] += self.h
                x1 = x0 + z1 + z2
                x2 = x0 + z1 - z2
                x3 = x0 - z1 + z2
                x4 = x0 - z1 - z2
                H[i,j] = (self.fun(x1) - self.fun(x2) - self.fun(x3) + self.fun(x4))/(4*(self.h**2))
        return H

    def defpos(self,Bk):
        return all(LA.eig(Bk)[0])>10e-2
        

    def mk(self,x0):
        G = grad(x0, self.h)
        H = hess(x0, self.h)
        norma = np.linalg.norm(G)
        p1 = -G/norma
        p = np.transpose(p1)
        mk = self.fun(x0) + p*G + 0.5*p*H*p1
        return mk



    def cholesky(self,Bk):      ##  modificador de la hessiana  
        maxIt=100
        beta = 10**-3
        if np.min(np.diag(Bk)) > 10e-2 :           
            factor = 0
        else:
            factor = -np.min(np.diag(Bk)) + beta
        for i in range(maxIt):
            Bk = Bk + factor*np.identity(len(Bk))
            

            if self.defpos(Bk):
                return factor
            else:
                factor = np.max([beta,2*factor])
        return factor
            




    def BLS(self, xk):         ##  Backtracking Line Search
        a = 1
        c =10e-4
        pk = -np.linalg.inv(self.hess(xk)).dot(self.grad(xk))
        iter = 1
        while self.fun(xk +a*pk) > self.fun(xk) + c*a*np.dot(np.transpose(self.grad(xk)),pk) and iter<20:
            a = a*.9
            iter= iter +1
            #print(iter)
        return a

    def newton(self,xk):
        maxIt=100          
        a = 1
        for k in range(maxIt):
            Bk = self.hess(xk)
            pk = np.linalg.solve(Bk,-self.grad(xk)) 
            xk = xk + a*pk           
            #print(k)
        return xk

    
    def newton_whm(self,xk):          ## Newton with hessian modification
        maxIt = 100
        n = len(xk)
        for k in range(maxIt):            
            Bk = self.hess(xk)
            
            factor = self.cholesky(Bk)
            Bk = self.hess(xk)+factor*np.identity(n)
            pk = LA.solve(Bk, -self.grad(xk))
            xk = xk + self.BLS(xk)*pk
        return xk


    

    def BFGS(self,xk):      ##BFGS
        tol = 10e-4
        Bk = self.hess(xk)
        Hk = LA.inv(Bk)
        while LA.norm(self.grad(xk)) > tol:
            gradf = self.grad(xk)
            pk = -Hk.dot(gradf)
            alpha = self.BLS(xk)
            aux = xk.copy()
            xk =  xk + alpha*pk
            sk = -aux+xk.copy()
            yk = self.grad(xk) - gradf
            rho = 1/(np.dot(yk.T,sk))
            U = np.identity(len(Hk))-(rho*sk).dot(np.transpose(yk))
            Hk1 = U.dot(Hk).dot(np.transpose(U)) + (rho*sk).dot(np.transpose(sk))
            x0 = xk
            gf = self.grad(xk)
            Hk = Hk1
        return xk

    def steepest_descent(self, x0):      ##Steepest descent en caso de que sea el ¡basic line search?
        for i in range(100):
            Bk = self.hess(x0)
            pk = np.linalg.solve(Bk,-self.grad(x0))
            x0 = x0 + self.BLS(x0)*pk
            #print(i)
        return x0

In [11]:
def Rosenbrock(x0):
    x0= x0.reshape(2,)
    a=1
    b=100
    x=x0[0]
    y=x0[1]
    f = (a-x)**2 + b*(y-x**2)**2
    return f

print("Función Rosenbrock\n")
a = optimization(Rosenbrock, 10e-5)

x0 = np.array([[3,2]]).T

print("El resultado obtenido mediante Steepest descent fue:")
print(a.basicLineSearch(x0))
print("\nEl resultado obtenido con la busqueda lineal de Newton:")
print(a.newton(x0))
print("\nEl resultado obtenido con la busqueda lineal de Newton modificada:")
print(a.newton_whm(x0))
print("\nEl resultado obtenido con BFGS:")
print(a.BFGS(x0))



Función Rosenbrock

El resultado obtenido mediante Steepest descent fue:
[[  7.84388852]
 [-58.00614781]]

El resultado obtenido con la busqueda lineal de Newton:
[[   31.85621786]
 [-1049.62250702]]

El resultado obtenido con la busqueda lineal de Newton modificada:
[[  7.84388852]
 [-58.00614781]]

El resultado obtenido con BFGS:
[[0.9714118]
 [0.9435922]]


In [3]:
###Misma clase con parametros distintos para poder correr el problema en la base de datos###


import numpy as np
from numpy import linalg as LA
import random as ran
    


class optimization:

    def __init__ (self,fun, h):
        self.fun = fun
        self.h = h

    def grad(self, x0):     ##gradiente
        n = len(x0)
        G = np.zeros((n,1),dtype = float)
        for i in range(n):
            xh = np.array(x0, dtype = float)
            xh[i,0] = xh[i,0] + self.h
            G[i,0] = round((self.fun(xh)-self.fun(x0))/self.h,3) 
        return G

    def hess(self, x0):     ##hessiana
        n = len(x0)
        x0 = x0.reshape((n,))
        s= (n,n)
        H = np.zeros(s)
        for i in range(n):
            for j in range(n):
                z1 = np.zeros(n)
                z2 = z1
                z1[i] += self.h
                z2[j] += self.h
                x1 = x0 + z1 + z2
                x2 = x0 + z1 - z2
                x3 = x0 - z1 + z2
                x4 = x0 - z1 - z2
                H[i,j] = (self.fun(x1) - self.fun(x2) - self.fun(x3) + self.fun(x4))/(4*(self.h**2))
        return H

    def defpos(self,Bk):
        return all(LA.eig(Bk)[0])>10e-1
        

    def mk(self,x0):
        G = grad(x0, self.h)
        H = hess(x0, self.h)
        norma = np.linalg.norm(G)
        p1 = -G/norma
        p = np.transpose(p1)
        mk = self.fun(x0) + p*G + 0.5*p*H*p1
        return mk



    def cholesky(self,Bk):      ##  modificador de la hessiana  
        maxIt=100
        beta = 10**-3
        if np.min(np.diag(Bk)) > 10e-2 :           
            factor = 0
        else:
            factor = -np.min(np.diag(Bk)) + beta
        for i in range(maxIt):
            Bk = Bk + factor*np.identity(len(Bk))
            

            if self.defpos(Bk):
                return factor
            else:
                factor = np.max([beta,2*factor])
        return factor
            



    def BLS(self, xk):         ##Backtracking Line Search
        a = 1
        c =10e-4
        try:
            pk = -np.linalg.inv(self.hess(xk)).dot(self.grad(xk))
        except:
            return a
        iter = 1
        while self.fun(xk +a*pk) > self.fun(xk) + c*a*np.dot(np.transpose(self.grad(xk)),pk) and iter<20:
            a = a*.9
            iter= iter +1
            #print(iter)
        return a

    def newton(self,xk):    ##Newton 
        maxIt=100          
        a = 1
        for k in range(maxIt):
            Bk = self.hess(xk)
            try:
                pk = np.linalg.solve(Bk,-self.grad(xk)) 
            except:
                return xk
            xk = xk + a*pk           
            #print(k)
        return xk

    
    def newton_whm(self,xk):          ## Newton with hessian modification
        maxIt = 100
        n = len(xk)
        for k in range(maxIt):            
            Bk = self.hess(xk)
            
            factor = self.cholesky(Bk)
            Bk = self.hess(xk)+factor*np.identity(n)
            try:
                pk = LA.solve(Bk, -self.grad(xk))
            except:
                return xk
            xk = xk + self.BLS(xk)*pk
        return xk


    

    def BFGS(self,xk):          ##BFGS
        tol = 10e-4
        Bk = self.hess(xk)
        Hk = LA.inv(Bk)
        while LA.norm(self.grad(xk)) > tol:
            gradf = self.grad(xk)
            pk = -Hk.dot(gradf)
            alpha = self.BLS(xk)
            aux = xk.copy()
            xk =  xk + alpha*pk
            sk = -aux + xk.copy()
            yk = self.grad(xk) - gradf
            rho = 1/(np.dot(yk.T,sk))
            U = np.identity(len(Hk))-(rho*sk).dot(np.transpose(yk))
            Hk1 = U.dot(Hk).dot(np.transpose(U)) + (rho*sk).dot(np.transpose(sk))
            x0 = xk
            gf = self.grad(xk)
            Hk = Hk1
        return xk

    def steepest_descent(self, x0):      ##Steepest descent: en caso de que este sea el ¿basic line search?
        for i in range(100):
            Bk = self.hess(x0)
            try:
                pk = np.linalg.solve(Bk,-self.grad(x0))
            except:
                return x0
            x0 = x0 + self.BLS(x0)*pk
            #print(i)
        return x0

In [4]:
import csv
file = open('crime_data.csv', 'r')
file.readline()
reader = csv.reader(file)
crimes = []
for crime in reader:
    crimes.append(crime[3:5])
camara_loc  = (np.random.rand(5,2) + [19, -100]).T
crime_loc = np.array(crimes,dtype="float")


def fcosto(camara_loc):
    camara_loc = camara_loc.reshape((5,2))
    #print(camara_loc)

    costo = 0
    for i in range(len(camara_loc)):
        for j in range(len(crime_loc)):
            costo += np.sqrt((camara_loc[i][0]-crime_loc[j][0])**2+(camara_loc[i][1]-crime_loc[j][1])**2)
        for k in range(len(camara_loc)):
            if i!=k:
                costo += 1/np.sqrt((camara_loc[i][0]-camara_loc[k][0])**2+(camara_loc[i][0]-camara_loc[k][0])**2)
    return costo


b = optimization(fcosto, 10e-5)

x0 = camara_loc.reshape((10,1))

print("Camaras (simplificado a 5 camaras y 300 crimenes)")
# print("El resultado obtenido mediante Steepest descent fue:")
# print(b.steepest_descent(x0))
# print("\nEl resultado obtenido con la busqueda lineal de Newton:")
# print(b.newton(x0).reshape((5,2)))
# print("\nEl resultado obtenido con la busqueda lineal de Newton modificada:")
# print(b.newton_whm(x0).reshape((5,2)))
print("\nEl resultado obtenido con BFGS:")
print(b.BFGS(x0).reshape((5,2)))

Camaras (simplificado a 5 camaras y 300 crimenes)

El resultado obtenido con BFGS:
[[ 5.31291635e+10  4.68084121e+13]
 [ 5.32246950e+10 -4.66761752e+13]
 [ 5.27719747e+10 -5.41660074e+09]
 [-7.55383772e+10 -6.70312386e+10]
 [-7.39865713e+10 -7.11301117e+10]]
